Copyright (c) Microsoft Corporation. All rights reserved.  
Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/aml-pipelines-setup-versioned-pipeline-endpoints.png)


# How to Setup a PipelineEndpoint and Submit a Pipeline Using the PipelineEndpoint.

In this notebook, we provide an introduction to Azure machine learning PipelineEndpoints. It covers:
* [Create PipelineEndpoint](#Create-PipelineEndpoint), How to create PipelineEndpoint.
* [Retrieving PipelineEndpoint](#Retrieving-PipelineEndpoint), How to get specific PipelineEndpoint from worskpace by name/Id and get all [PipelineEndpoints](#Get-all-PipelineEndpoints-in-workspace) within workspace.
* [PipelineEndpoint Properties](#PipelineEndpoint-properties). How to get and set PipelineEndpoint properties, such as default version of PipelineEndpoint.
* [PipelineEndpoint Submission](#PipelineEndpoint-Submission). How to run a Pipeline using PipelineEndpoint.

### Prerequisites

* Install azure cli with azure-cli-ml extension following the [instructions here](setup-environment.ipynb).

### Setup workspace
Login to azure with cli and set the default workspace using `az ml folder attach` command.

After this operation, the workspace could be retrived with the `Workspace.from_config()` for SDK usage.

In [ ]:
# NOTE: Update the following information with your environment

SUBSCRIPTION_ID = '<your subscription ID>'
WORKSPACE_NAME = '<your workspace name>'
RESOURCE_GROUP_NAME = '<your resource group>'

In [ ]:
!az login -o none 
!az account set -s $SUBSCRIPTION_ID 
!az ml folder attach -w $WORKSPACE_NAME -g $RESOURCE_GROUP_NAME 

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()

###  Create PipelineEndpoint
Following are required input parameters to create PipelineEndpoint:

* *workspace*: AML workspace.
* *name*: name of PipelineEndpoint, it is unique within workspace.
* *description*: description details for PipelineEndpoint.
* *pipeline*: A Pipeline or PipelineRun, to set default version of PipelineEndpoint.
* *set_as_default*: whether set current *pipeline* as the default pipeline for created PipelineEndpoint.

#### Initialization, Steps to create a Pipeline


In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Dataset
from azureml.pipeline.wrapper import Module, Pipeline

# Define a single step pipeline for demonstration purpose.
def single_node_pipeline(workspace):
    """
    Define a single step pipeline for demonstration purpose.

    :param workspace: The workspace the published pipeline was created in.
    :type workspace: azureml.core.Workspace
    """
    # Retrieve an already attached Azure Machine Learning Compute.
    cluster_name = "cpu-cluster"
    try:
        compute_target = ComputeTarget(workspace=workspace, name=cluster_name)
        print('Found existing compute target {}.'.format(cluster_name))
    except ComputeTargetException:
        print('Creating a new compute target...')
        compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_v2",
                                                               max_nodes=4)

        compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
        compute_target.wait_for_completion(show_output=True, timeout_in_minutes=20)

    print("Azure Machine Learning Compute attached")

    # load Module
    execute_python_script_module = Module.load(workspace, namespace='azureml', name='Execute Python Script')

    # get dataset
    training_data_name = 'aml_module_training_data'

    if training_data_name not in workspace.datasets:
        print('Registering a training dataset for sample pipeline ...')
        train_data = Dataset.File.from_files(path=['https://dprepdata.blob.core.windows.net/demo/Titanic.csv'])
        train_data.register(workspace=workspace,
                            name=training_data_name,
                            description='Training data (just for illustrative purpose)')
        print('Dataset registered')

    train_data = Dataset.get_by_name(workspace, name=training_data_name)

    # define a module
    print('Creating a Module for sample pipeline ...')
    module = execute_python_script_module(dataset1=train_data)
    print("Module created")

    # define a pipeline
    pipeline = Pipeline(nodes=[module],
                        workspace=workspace,
                        name="test-pipeline",
                        description="test pipeline for pipeline-endpoint test",
                        default_compute_target=cluster_name)
    print("Pipeline is built")

    return pipeline
pipeline = single_node_pipeline(workspace=ws)

#### Create a PipelineRun
A PipelineRun can be created by submitting a Pipeline.

In [ ]:
experiment_name = "PipelineEndpointTestExperiment"

pipeline_run = pipeline.submit(experiment_name=experiment_name)

#### Publish PipelineEndpoint
Create PipelineEndpoint using Pipeline with required parameters: workspace, name, description, pipeline and set_as_default.

In [ ]:
from azureml.pipeline.wrapper import PipelineEndpoint

pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name="PipelineEndpointTest",
                                             pipeline=pipeline, description="Test description Notebook", 
                                             set_as_default=True)

pipeline_endpoint

Create PipelineEndpoint using PipelineRun with required parameters: workspace, name, description, pipeline and set_as_default.

In [ ]:
pipeline_endpoint = PipelineEndpoint.publish(workspace=ws, name="PipelineEndpointTest",
                                             pipeline=pipeline_run, description="Test description Notebook", 
                                             set_as_default=True)

pipeline_endpoint

### Retrieving PipelineEndpoint

PipelineEndpoint is uniquely defined by name and id within workspace. PipelineEndpoint in workspace can be retrived by Id or by name.

#### Get PipelineEndpoint by Name



In [ ]:
pipeline_endpoint_by_name = PipelineEndpoint.get(workspace=ws, name="PipelineEndpointTest")
pipeline_endpoint_by_name

#### Get PipelineEndpoint by Id


In [ ]:
#get the PipelineEndpoint Id
pipeline_endpoint_by_name = PipelineEndpoint.get(workspace=ws, name="PipelineEndpointTest")
endpoint_id = pipeline_endpoint_by_name.id

pipeline_endpoint_by_id = PipelineEndpoint.get(workspace=ws, id=endpoint_id)
pipeline_endpoint_by_id

#### Get all PipelineEndpoints in workspace
Returns all PipelineEndpoints within a workspace, 

In [ ]:
endpoint_list = PipelineEndpoint.list(workspace=ws, active_only=True)
endpoint_list

### PipelineEndpoint properties

#### Default Version of PipelineEndpoint
Default version of PipelineEndpoint starts from "0" and increments on addition of pipelines.

####  Set default version 

In [ ]:
pipeline_endpoint_by_name.set_default_version(version="0")

#### Get all pipeline versions in PipelineEndpoint
Returns a dictionary of published pipelines and its versions, with format {version: pipeline}.

*active_only* parameter dicedes whether to return only active pipelines.

In [ ]:
pipelines_list = pipeline_endpoint_by_name.list_pipelines(active_only=True)

pipelines_list

#### Change PipelineEndpoint status

##### Disable PipelineEndpoint

In [ ]:
pipeline_endpoint_by_name.disable()
print(pipeline_endpoint_by_name.status)

##### Enable PipelineEndpoint

In [ ]:
pipeline_endpoint_by_name.enable()
print(pipeline_endpoint_by_name.status)

### PipelineEndpoint Submission
PipelineEndpoint triggers specific versioned pipeline or default pipeline by:
* Submit call
* Rest Endpoint

#### Run Pipeline by Submit call of PipelineEndpoint 
Run default pipeline using Submit api of PipelineEndpoint

In [ ]:
run_submitted_by_pipeline_endpoint = pipeline_endpoint_by_name.submit(experiment_name="test_submit_by_pipeline_endpoint")

run_submitted_by_pipeline_endpoint

#### Run Pipeline by endpoint property of PipelineEndpoint
Run pipeline using endpoint property of PipelineEndpoint and executing http post.

In [ ]:
# endpoint with id 
rest_endpoint_id = pipeline_endpoint_by_name.endpoint

# for default version pipeline
rest_endpoint_id_without_version_with_id = rest_endpoint_id

# for specific version pipeline just append version info
version="0"
rest_endpoint_id_with_version = rest_endpoint_id_without_version_with_id+"/"+ version

print(rest_endpoint_id_with_version)

In [ ]:
# endpoint with name
rest_endpoint_name = rest_endpoint_id.split("Id", 1)[0] + "Name?name=" + pipeline_endpoint_by_name.name

# for default version pipeline
rest_endpoint_name_without_version = rest_endpoint_name

# for specific version pipeline just append version info
version="0"
rest_endpoint_name_with_version = rest_endpoint_name_without_version+"&pipelineVersion="+ version

print(rest_endpoint_name_with_version)

[This notebook](https://aka.ms/pl-restep-auth) shows how to authenticate to AML workspace.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication
import requests

auth = InteractiveLoginAuthentication()
aad_token = auth.get_authentication_header()

print("You can perform HTTP POST on URL {} to trigger this pipeline".format(rest_endpoint_name_with_version))

# specify the param when running the pipeline, post uro can both be rest_endpoint_name_with_version and rest_endpoint_id_with_version
response = requests.post(rest_endpoint_name_with_version, 
                         headers=aad_token, 
                         json={"ExperimentName": "default_pipeline",
                               "RunSource": "SDK",
                               "ParameterAssignments": {"1": "united", "2":"city"}})
# or
# response = requests.post(rest_endpoint_id_with_version, 
#                          headers=aad_token, 
#                          json={"ExperimentName": "default_pipeline",
#                                "RunSource": "SDK",
#                                "ParameterAssignments": {"1": "united", "2":"city"}})

In [ ]:
try:
    response.raise_for_status()
except Exception:    
    raise Exception('Received bad response from the endpoint: {}\n'
                    'Response Code: {}\n'
                    'Headers: {}\n'
                    'Content: {}'.format(rest_endpoint_name_with_version, response.status_code, response.headers, response.content))

run_id = response.json().get('Id')

print('Submitted pipeline run: ', run_id)